# Web Scraping from Indeed Website

* Log: 02082017 Jiewei Chen Da Xue
* Log: 02102016 Jiewei Chen Da Xue Nick Ulle
* Log: 02212017 Jiewei Chen
* Log: 02262017 Jiewei Chen Da Xue
* Log: 02272017 Jiewei Chen Da Xue
* Log: 02282017 Jiewei Chen Da Xue

## Step 1. Using API get 600 Posts

In [2]:
import requests
import requests_cache
requests_cache.install_cache("cache")
import pandas as pd
from bs4 import BeautifulSoup


import string
import nltk
#from nltk import corpus
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer   # sklearn --- primer machine learning package
from sklearn.neighbors import NearestNeighbors

In [20]:
publisher_id = "254802918413674"

http://api.indeed.com/ads/apisearch?publisher=254802918413674&q=java&l=austin%2C+tx&sort=&radius=&st=&jt=&start=&limit=&fromage=&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2

One example of: url = "http://api.indeed.com/ads/apisearch?publisher=254802918413674&format=json&q=data&l=sunnyvale%2C+ca&sort=&radius=0&st=&jt=internship&start=&limit=1000&fromage=30&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2"

In [75]:
def get_indeed(city, state, keyword, radius, jobtype = "fulltime", js = "json", start = 0, publisher_id = publisher_id):
    url = "http://api.indeed.com/ads/apisearch?"
    if city == "":
        location = state
    else: 
        location = city + ', ' + state
    result = requests.get(url, params = {
            "q": keyword,
            "v": 2,
            "format": js,
            "publisher": publisher_id,
            "l": location,
            "jt": jobtype, # "fulltime", "parttime", "contract", "internship", "temporary"
            "radius": radius,
            "start": start,
            "limit": 100,
            "latlong": 1})
    result = result.json()['results']
    return result

In [92]:
# This is a data frame containing basic information of job posts
job_list = pd.DataFrame()
for i in range(0,651,25):
    temp = pd.DataFrame(get_indeed('','ca', '"data analyst"', 0, start = i))
    job_list = job_list.append(temp)


Save Result to a csv file, so that next time we run this code it will be faster. Don't need to reload from website.

In [97]:
print(job_list.shape)
job_list.head()

(663, 21)


,Unnamed: 0,city,company,country,date,expired,formattedLocation,formattedLocationFull,formattedRelativeTime,indeedApply,...,jobtitle,latitude,longitude,onmousedown,snippet,source,sponsored,state,stations,url
0,0,Palo Alto,Visa,US,"Fri, 24 Feb 2017 04:08:33 GMT",False,"Palo Alto, CA","Palo Alto, CA",4 days ago,False,...,Data Analyst,37.439560,-122.14286,"indeed_clk(this,'1411');",Acquire and integrate <b>data</b> from multipl...,Visa,False,CA,NaN,http://www.indeed.com/viewjob?jk=94c1d7db6e730...
1,1,San Francisco,Microsoft,US,"Sat, 18 Feb 2017 12:46:21 GMT",False,"San Francisco, CA","San Francisco, CA",10 days ago,False,...,Data Analyst,37.774727,-122.41758,"indeed_clk(this,'1411');","Manipulate and analyze complex, high-volume, h...",Microsoft,False,CA,NaN,http://www.indeed.com/viewjob?jk=21d944e4f8585...
2,2,Irvine,Academic Organization,US,"Tue, 14 Feb 2017 23:11:45 GMT",False,"Irvine, CA","Irvine, CA",13 days ago,True,...,Data Analysis - Entry level,33.667583,-117.81868,"indeed_clk(this,'1411');",An academic organization in Irvine is currentl...,Indeed,False,CA,NaN,http://www.indeed.com/viewjob?jk=3ce45e48d1d03...
3,3,San Diego,PCS Global Tech,US,"Thu, 16 Feb 2017 15:04:02 GMT",False,"San Diego, CA","San Diego, CA",12 days ago,True,...,Full Time - Entry Level SQL/BI Position,32.714287,-117.15385,"indeed_clk(this,'1411');",Knowledge of BI tools or <b>data</b> warehouse...,Indeed,False,CA,NaN,http://www.indeed.com/viewjob?jk=1fa7d18051c75...
4,4,San Francisco,Blend Labs,US,"Tue, 24 Jan 2017 02:42:24 GMT",False,"San Francisco, CA","San Francisco, CA 94103",30+ days ago,False,...,Data Analyst,37.776352,-122.41416,"indeed_clk(this,'1411');",The <b>Data</b> <b>Analyst</b> will support <b...,Blend Labs,False,CA,NaN,http://www.indeed.com/viewjob?jk=3a266930712ad...


In [94]:
# save result into a csv file
job_list.to_csv('job_ca_dataanalyst.csv')

In [95]:
# read csv
job_list = pd.read_csv('job_ca_dataanalyst.csv',index_col=False)

In [96]:
# check csv
print(job_list.shape)
job_list.head()

(663, 21)


,Unnamed: 0,city,company,country,date,expired,formattedLocation,formattedLocationFull,formattedRelativeTime,indeedApply,...,jobtitle,latitude,longitude,onmousedown,snippet,source,sponsored,state,stations,url
0,0,Palo Alto,Visa,US,"Fri, 24 Feb 2017 04:08:33 GMT",False,"Palo Alto, CA","Palo Alto, CA",4 days ago,False,...,Data Analyst,37.439560,-122.14286,"indeed_clk(this,'1411');",Acquire and integrate <b>data</b> from multipl...,Visa,False,CA,NaN,http://www.indeed.com/viewjob?jk=94c1d7db6e730...
1,1,San Francisco,Microsoft,US,"Sat, 18 Feb 2017 12:46:21 GMT",False,"San Francisco, CA","San Francisco, CA",10 days ago,False,...,Data Analyst,37.774727,-122.41758,"indeed_clk(this,'1411');","Manipulate and analyze complex, high-volume, h...",Microsoft,False,CA,NaN,http://www.indeed.com/viewjob?jk=21d944e4f8585...
2,2,Irvine,Academic Organization,US,"Tue, 14 Feb 2017 23:11:45 GMT",False,"Irvine, CA","Irvine, CA",13 days ago,True,...,Data Analysis - Entry level,33.667583,-117.81868,"indeed_clk(this,'1411');",An academic organization in Irvine is currentl...,Indeed,False,CA,NaN,http://www.indeed.com/viewjob?jk=3ce45e48d1d03...
3,3,San Diego,PCS Global Tech,US,"Thu, 16 Feb 2017 15:04:02 GMT",False,"San Diego, CA","San Diego, CA",12 days ago,True,...,Full Time - Entry Level SQL/BI Position,32.714287,-117.15385,"indeed_clk(this,'1411');",Knowledge of BI tools or <b>data</b> warehouse...,Indeed,False,CA,NaN,http://www.indeed.com/viewjob?jk=1fa7d18051c75...
4,4,San Francisco,Blend Labs,US,"Tue, 24 Jan 2017 02:42:24 GMT",False,"San Francisco, CA","San Francisco, CA 94103",30+ days ago,False,...,Data Analyst,37.776352,-122.41416,"indeed_clk(this,'1411');",The <b>Data</b> <b>Analyst</b> will support <b...,Blend Labs,False,CA,NaN,http://www.indeed.com/viewjob?jk=3a266930712ad...


## Problems

<ol>
    <li> Can only get at max 25 jobs per request <font color="red">Change start params</font>
    <li> When search for "Data Analyst", it is actually searching for "Data" and "Analyst" as two seperated key words
    <li> Cannot extract the job requirement information <font color="red"> Probably need web scraping </font>
    <li> 
</ol>

## Step 2. Webscraping each post

In [98]:
url_list = list(job_list['url'])

In [120]:
def get_post_details(url):
    """
    This function extracts information of one post by taking a url as input
    Output is a dictionary with job title, company name, and description of the job.
    """
    post_doc = requests.get(url).content
    post = BeautifulSoup(post_doc, 'html.parser')
    try:
        title = post.find('b',attrs={'class':'jobtitle'}).text
    except AttributeError:
        title = ''
    try:
        company = post.find(attrs={'class':'company'}).text
    except AttributeError:
        company = ''
        
    dis_html = post.find(attrs={'id':'job_summary'})
    #dis_html = post.find(attrs={'class':'snip'})
    
    post_dict = {'title':title, 'company':company, 'description':dis_html}
    return post_dict

In [121]:
# use `job_summary` to extract
post_summary = [get_post_details(url) for url in url_list]
post_summary = pd.DataFrame(post_summary )
post_summary.tail()

,company,description,title
658,Redfish Technology - High Tech Executive Recru...,"<span class=""summary"" id=""job_summary"">#22530 ...","SENIOR TECHNICAL PRODUCT MANAGER, SENIOR TECHN..."
659,Connexity,"<span class=""summary"" id=""job_summary"">Connexi...",Sr. Data Scientist-Machine Learning Engineer
660,Handshake,"<span class=""summary"" id=""job_summary"">Our mis...",Data Analyst
661,Visual Supply Co. (VSCO),"<span class=""summary"" id=""job_summary"">You are...",Senior Data Analyst
662,Scopely,"<span class=""summary"" id=""job_summary"">Are you...",Lead Data Analyst - SQL - Tableau


In [122]:
# check how many post without job description
m = 0
test = list(post_summary['description'])
for i in range(len(test3)):
    if subtitles[i]:
        m += 1
print (m/len(test))

0.6063348416289592


In [ ]:
# This `post_summary` is a data frame containing 'job title', 'company', 
#      and 'job description' extracted from website of each post
# post_summary = [get_post_details(url) for url in url_list]
# post_summary = pd.DataFrame(post_summary )

In [101]:
# post_summary.tail()

,company,description,title
658,Redfish Technology - High Tech Executive Recru...,"<td class=""snip""> <span class=""summary"" id=""jo...","SENIOR TECHNICAL PRODUCT MANAGER, SENIOR TECHN..."
659,Connexity,"<td class=""snip""> <span class=""summary"" id=""jo...",Sr. Data Scientist-Machine Learning Engineer
660,Handshake,"<td class=""snip""> <span class=""summary"" id=""jo...",Data Analyst
661,Visual Supply Co. (VSCO),"<td class=""snip""> <span class=""summary"" id=""jo...",Senior Data Analyst
662,Scopely,"<td class=""snip""> <span class=""summary"" id=""jo...",Lead Data Analyst - SQL - Tableau


In [129]:
post_summary.to_csv('postsummary_3.csv')

#### Check how many missing values

In [130]:
post_summary.isnull().sum()

company        0
description    0
title          0
dtype: int64

## Step 3.  Extracting Important Skills (without using subtitles)

In [ ]:
stemmer = PorterStemmer().stem

tokenize = nltk.word_tokenize

def stem(tokens,stemmer = PorterStemmer().stem):
    return [stemmer(w.lower()) for w in tokens] 

def lemmatize(text):
    """
    Extract simple lemmas based on tokenization and stemming
    Input: string
    Output: list of strings (lemmata)
    """
    return stem(tokenize(text))

stop = set(stopwords.words('english'))

In [199]:
def diction_qual(files):
    """
    Returns a dictionary from lemmata to document ids containing that lemma
    Input is a list of job description text
    Output is a dictionary with lemmata as key and document ids as values
    """
    textd = {} 
    for i in range(len(files)):
        # loop over each raw text
        t = files[i]
        # return unique and order list of words appeared in the raw text
        s = set(lemmatize(t))- stop - set(string.punctuation)
        try:
            toks = toks | s   # append to "toks" set a "s"
        except NameError:
            toks = s    # if doesn't exsit, initialize it
        for tok in s:
            try:
                textd[tok].append(i)
            except KeyError:
                textd[tok] = [i]
    
    return textd

In [200]:
job_des = list(post_summary['description'])
job_text = [job.text for job in job_des]

# Lemmata of subtitles and the id of posts containing that Lemmata
text_id =diction_qual(job_text)

# Lemmata of subtitles and the number of posts containing that Lemmata
text_num = {key:len(set(val)) for key,val in text_id.items()}
# Sort text_num
sorted_text = [(key, text_num[key]) for key in sorted(text_num, key=lambda x : text_num[x],
                                                              reverse=True)]
df_sorted_text = pd.DataFrame(sorted_text)
df_sorted_text.columns = ['word', '# of posts']
df_sorted_text.iloc[:50,:]

,word,# of posts
0,develop,424
1,support,351
2,use,345
3,degre,326
4,analyz,315
5,process,312
6,project,304
7,strong,290
8,relat,287
9,system,284


## Step 3.  Extracting Important Skills (using subtitles)
### Strategy for Extracting Qualifications
<ol>
    <li> compare all subtitles and find out which subtitles are "Qualifications"
    <li> split text by using subtitles
    <li> compare the word in qualifications and analyze
    
</ol>

### Compare all subtitles and find out which subtitles are "Qualifications"

In [193]:
# get subtitles inside job description
def get_subtitle(post):
    """
    This function takes the post of each job and get all subtitles of the post
    Input: post in html format
    Output: a list of subtitles
    """
    try:
        temp = post.find_all('b')
        subtitles = [sub.text for sub in temp]
    except AttributeError:
        subtitles = []
    return subtitles

# a list containing the subtitles of each post
subtitles = [get_subtitle(post) for post in list(post_summary['description'])]

In [194]:
# Count how many posts don't have subtitles
empty_subtitles = 0
for sub in subtitles:
    if not sub:        
        empty_subtitles += 1
empty_subtitles

261

In [133]:
def diction(files):
    """
    Returns a dictionary from lemmata to document ids containing that lemma
    Input is a data frame
    Output is a dictionary with lemmata as key and document ids as values
    """
    textd = {} 
    for i in range(len(files)):
        # loop over each raw text
        t = ' '.join(files[i])
        # return unique and order list of words appeared in the raw text
        s = set(lemmatize(t))- stop - set(string.punctuation)
        try:
            toks = toks | s   # append to "toks" set a "s"
        except NameError:
            toks = s    # if doesn't exsit, initialize it
        for tok in s:
            try:
                textd[tok].append(i)
            except KeyError:
                textd[tok] = [i]
    
    return textd

In [195]:
# Lemmata of subtitles and the id of posts containing that Lemmata
sub_id = diction(subtitles)

In [196]:
# Lemmata of subtitles and the number of posts containing that Lemmata
sub_num = {key:len(set(val)) for key,val in sub_id.items()}
# Sort sub_num
sorted_sub = [(key, sub_num[key]) for key in sorted(sub_num, key=lambda x : sub_num[x],
                                                              reverse=True)]
df_sorted_sub = pd.DataFrame(sorted_sub)
df_sorted_sub.columns = ['word', '# of posts']
df_sorted_sub.iloc[:50,:]

,word,# of posts
0,respons,163
1,qualif,138
2,requir,117
3,experi,84
4,skill,75
5,job,66
6,educ,65
7,essenti,52
8,duti,49
9,prefer,48


In [204]:
list(df_sorted_sub['word'])[:100]

['respons',
 'qualif',
 'requir',
 'experi',
 'skill',
 'job',
 'educ',
 'essenti',
 'duti',
 'prefer',
 'posit',
 'data',
 'summari',
 'work',
 'analyst',
 'knowledg',
 'function',
 'role',
 'descript',
 'minimum',
 'benefit',
 'includ',
 'abil',
 'primari',
 'locat',
 'thi',
 'candid',
 'applic',
 'us',
 'physic',
 'report',
 'titl',
 'need',
 'opportun',
 'compani',
 'employ',
 'appli',
 'key',
 'technic',
 'addit',
 'success',
 'commun',
 'level',
 "'s",
 'condit',
 'offer',
 'time',
 'you’ll',
 'get',
 'note',
 'ideal',
 'inform',
 'bonu',
 'excel',
 'compet',
 'analyt',
 'pleas',
 'manag',
 'perk',
 'pay',
 'team',
 'consider',
 'overview',
 'equal',
 'area',
 'analysi',
 'environ',
 'desir',
 'ca',
 '–',
 'comput',
 'employe',
 'special',
 'year',
 'durat',
 'follow',
 'date',
 'servic',
 'demand',
 'orient',
 'licens',
 'perform',
 'career',
 'hour',
 'must',
 'remot',
 'statu',
 'start',
 'resum',
 'train',
 'and/or',
 'busi',
 'profession',
 'keyword',
 'qualifi',
 'specif',


In [197]:
# add subtitles to the 
stop_require = set(list(df_sorted_sub['word'])[:100])

In [198]:
'sql' in stop_require

False

### Split text by using subtitles

In [137]:
keywords = list(df_sorted_sub['word'][[1,2,4]])

def get_qual(subtitles):
    """
    This function takes the subtitles and check which subtitle is a subtitle of qualifications
    Input: a list of subtitles
    Output: subtitles of qualifications
    """
    qual = [sub for sub in subtitles if set(lemmatize(sub)).intersection(keywords)]
    return qual

qual_sub = [get_qual(sub) for sub in subtitles]

In [141]:
# Count how many posts don't have qualification subtitles
empty_qual_sub = 0
for qual in qual_sub:
    if not qual:        
        empty_qual_sub += 1
empty_qual_sub

415

### Compare the word in qualifications and analyze

In [142]:
def qualif_text(text, subtitles):
    '''This function cleans the text file and returns only the part related to skill/qualifications
    Input:  text ------- job description of a job post
            subtitles -- subtitles related to skill/qualifications of the post 
    Output: a str object of the essence of the job description
    '''
    result = text.split(subtitles[0])[1:]
    return result

In [143]:
# Extract text that contains qualifications
qual_text = []
job_des = list(post_summary['description'])
for i in range(len(qual_sub)):
    try:
        job_text = job_des[i].text
    except AttributeError:
        job_text = ''
    try:
        qual_text = qual_text + qualif_text(job_text, qual_sub[i])
    except IndexError:
        qual_text = qual_text + [job_text]


In [167]:
qual_keyw_id = diction_qual(qual_text)

In [168]:
qual_keyw_num = {key:len(set(val)) for key,val in qual_keyw_id.items()}
# Sort sub_num
sorted_qual_keyw = [(key, qual_keyw_num[key]) for key in sorted(qual_keyw_num, key=lambda x : qual_keyw_num[x],
                                                              reverse=True)]
df_sorted_qual = pd.DataFrame(sorted_qual_keyw)
df_sorted_qual.columns = ['word', '# of posts']
df_sorted_qual.iloc[:50,:]

,word,# of posts
0,complex,169
1,collabor,160
2,scienc,158
3,well,144
4,larg,138
5,verbal,132
6,implement,130
7,microsoft,122
8,queri,118
9,access,118


## Step 4.  Degree Requirement

In [162]:
def degree_require(text):
    '''This function finds whether graduate degree is required in a job description
    Output: scaler of 1 --- requires graduate degree
                      0 --- no requirement for graduate degree
    '''
    kw = ['graduate', 'm.s.', "master's"]
    result = 'undergrad'
    for word in kw:
        if word in text:
            result = 'graduate'
            break
        break
    return result

In [163]:
degree_list = []
for job in job_des:
    #try:
    degree_list = degree_list + [degree_require(job.text)]
    #except AttributeError:
        #degree_list = degree_list + []

In [164]:
# Count how many posts don't have degree requirement
empty_degree = 0
for d in degree_list:
    if not d:        
        empty_degree += 1
empty_degree

0

# Random Test